In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder.appName("app").getOrCreate() 

events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv") \
                    .withColumn("event_date", expr("date_trunc('day', event_time)"))

events.show()

25/06/28 04:20:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|2021-09-26 00:00:00|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|2021-02-21 00:00:00|
| -824540328| 532630305|    NULL|admi

In [4]:
devices = spark.read.option("header", "true").csv("/home/iceberg/data/devices.csv")

devices.show()

+-----------+--------------------+-------+------------------+
|  device_id|        browser_type|os_type|       device_type|
+-----------+--------------------+-------+------------------+
|-2147042689|             Firefox| Ubuntu|             Other|
|-2146219609|            WhatsApp|  Other|            Spider|
|-2145574618|       Chrome Mobile|Android|Generic Smartphone|
|-2144707350|Chrome Mobile Web...|Android|  Samsung SM-G988B|
|-2143813999|Mobile Safari UI/...|    iOS|            iPhone|
|-2142634982|   Chrome Mobile iOS|    iOS|            iPhone|
|-2142350383|   Chrome Mobile iOS|    iOS|            iPhone|
|-2141256237|   Chrome Mobile iOS|    iOS|            iPhone|
|-2138977887|             Firefox|OpenBSD|             Other|
|-2136667425|              Chrome|Windows|             Other|
|-2136444196|   Chrome Mobile iOS|    iOS|            iPhone|
|-2136415223|Chrome Mobile Web...|Android| Samsung SM-A305GT|
|-2136251094|             Firefox| Ubuntu|             Other|
|-213605

In [5]:
df = events.join(devices, on="device_id", how="left")

df.show(5)

+---------+-----------+--------+--------------------+---+--------------------+-------------------+------------+-------+-----------+
|device_id|    user_id|referrer|                host|url|          event_time|         event_date|browser_type|os_type|device_type|
+---------+-----------+--------+--------------------+---+--------------------+-------------------+------------+-------+-----------+
|532630305| 1037710827|    NULL| www.zachwilson.tech|  /|2021-03-08 17:27:...|2021-03-08 00:00:00|       Other|  Other|      Other|
|532630305|  925588856|    NULL|    www.eczachly.com|  /|2021-05-10 11:26:...|2021-05-10 00:00:00|       Other|  Other|      Other|
|532630305|-1180485268|    NULL|admin.zachwilson....|  /|2021-02-17 16:19:...|2021-02-17 00:00:00|       Other|  Other|      Other|
|532630305|-1044833855|    NULL| www.zachwilson.tech|  /|2021-09-24 15:53:...|2021-09-24 00:00:00|       Other|  Other|      Other|
|532630305|  747494706|    NULL| www.zachwilson.tech|  /|2021-09-26 16:03:..

In [6]:
df = df.withColumnsRenamed({"browser_type": "browser_family", "os_type": "os_family"})

df.show(5)

+---------+-----------+--------+--------------------+---+--------------------+-------------------+--------------+---------+-----------+
|device_id|    user_id|referrer|                host|url|          event_time|         event_date|browser_family|os_family|device_type|
+---------+-----------+--------+--------------------+---+--------------------+-------------------+--------------+---------+-----------+
|532630305| 1037710827|    NULL| www.zachwilson.tech|  /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|      Other|
|532630305|  925588856|    NULL|    www.eczachly.com|  /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|      Other|
|532630305|-1180485268|    NULL|admin.zachwilson....|  /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|      Other|
|532630305|-1044833855|    NULL| www.zachwilson.tech|  /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|    Other|      Other|
|532630305|  747494706|    NULL| www.zachwilson.

In [7]:
sorted_df = df.repartition(10, col("event_date")) \
                .sortWithinPartitions(col("event_date"), col("host")) \
                .withColumn("event_time", col("event_time").cast("timestamp"))

# global sort
sorted_df2 = df.repartition(10, col("event_date")) \
                .sort(col("event_date"), col("host")) \
                .withColumn("event_time", col("event_time").cast("timestamp"))

sorted_df.show(5)
sorted_df2.show(5)

+----------+-----------+--------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
| device_id|    user_id|referrer|                host|                 url|          event_time|         event_date|browser_family|os_family|       device_type|
+----------+-----------+--------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
| 532630305| 1129583063|    NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|         Other|    Other|             Other|
|1088283544| -648945006|    NULL|    www.eczachly.com|                   /|2021-01-07 02:58:...|2021-01-07 00:00:00|      PetalBot|  Android|Generic Smartphone|
|-158310583|-1871780024|    NULL|    www.eczachly.com|                   /|2021-01-07 04:17:...|2021-01-07 00:00:00|      PetalBot|    Other|            Spider|
|1088283544|  203689086|    NULL| 

[Stage 13:=============================>                            (2 + 2) / 4]

+----------+----------+--------+--------------------+-------------+--------------------+-------------------+--------------+---------+-----------+
| device_id|   user_id|referrer|                host|          url|          event_time|         event_date|browser_family|os_family|device_type|
+----------+----------+--------+--------------------+-------------+--------------------+-------------------+--------------+---------+-----------+
|-643696601|1272828233|    NULL|admin.zachwilson....|            /|2021-01-02 13:53:...|2021-01-02 00:00:00|        Chrome|  Windows|      Other|
| 532630305| 747494706|    NULL|admin.zachwilson....|            /|2021-01-02 19:36:...|2021-01-02 00:00:00|         Other|    Other|      Other|
| 898871897|2110046626|    NULL|admin.zachwilson....|/wp-login.php|2021-01-02 19:57:...|2021-01-02 00:00:00|        Chrome|    Linux|      Other|
|-643696601|1272828233|    NULL|admin.zachwilson....|            /|2021-01-02 21:05:...|2021-01-02 00:00:00|        Chrome| 

In [8]:
# .sortWithinPartitions() sorts within partitions, whereas .sort() is a global sort, which is very slow

# Note - exchange is synonymous with Shuffle

sorted_df.explain()
sorted_df2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#244, event_date#29, browser_family#181, os_family#182, device_type#93]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=529]
         +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#91 AS browser_family#181, os_type#92 AS os_family#182, device_type#93]
            +- BroadcastHashJoin [device_id#18], [device_id#90], LeftOuter, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilt

In [11]:
%%sql
create database if not exists bootcamp

++
||
++
++

In [12]:
%%sql
create table if not exists bootcamp.events (
    url string,
    referrer string,
    browser_family string,
    os_family string,
    device_family string,
    host string,
    event_time timestamp,
    event_date date
) using iceberg partitioned by (years(event_date))

++
||
++
++

In [16]:
%%sql
select * from bootcamp.events

url,referrer,browser_family,os_family,device_family,host,event_time,event_date


In [17]:
%%sql
create table if not exists bootcamp.events_sorted (
    url string,
    referrer string,
    browser_family string,
    os_family string,
    device_family string,
    host string,
    event_time timestamp,
    event_date date
) using iceberg partitioned by (years(event_date))

++
||
++
++

In [26]:
%%sql
create table if not exists bootcamp.events_unsorted (
    url string,
    referrer string,
    browser_family string,
    os_family string,
    device_family string,
    host string,
    event_time timestamp,
    event_date date
) using iceberg partitioned by (years(event_date))

++
||
++
++

In [33]:
start_df = df.repartition(4, col("event_date")) \
                .withColumn("event_time", col("event_time").cast("timestamp"))

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host"), col("browser_family"))

df.write.mode("overwrite").saveAsTable("bootcamp.events")
start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [28]:
%%sql
select sum(file_size_in_bytes) as size, count(1) as num_files, 'sorted'
from bootcamp.events_sorted.files
union all
select sum(file_size_in_bytes) as size, count(1) as num_files, 'unsorted'
from bootcamp.events_unsorted.files

size,num_files,sorted
5117103,4,sorted
5556664,4,unsorted


In [34]:
%%sql
select sum(file_size_in_bytes) as size, count(1) as num_files
from bootcamp.events.files

size,num_files
6029732,4
